# Modelos

In [ ]:
!pip install tensorflow

In [ ]:
!pip install silence_tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

In [ ]:
import silence_tensorflow.auto

In [ ]:
from keras.utils.image_dataset import image_dataset_from_directory
url = 'E:/CRM/UL/5/foto mono/Cotton plant disease'
dataset = image_dataset_from_directory(url)

In [ ]:
class_names = dataset.class_names
print(class_names)

In [ ]:
from keras.utils import image_dataset
train_set = image_dataset_from_directory(
    url,
    validation_split = 0.3,
    subset = "training",
    seed = 123,
    image_size = (256, 256),
    batch_size = 32
)

In [ ]:
test_set = image_dataset_from_directory(
    url,
    validation_split = 0.15,
    subset = "validation",
    seed = 123,
    image_size = (256, 256),
    batch_size = 32
)

In [ ]:
val_set = image_dataset_from_directory(
    url,
    validation_split = 0.15,
    subset = "validation",
    seed = 123,
    image_size = (256, 256),
    batch_size = 32
)

In [ ]:
#Melhoria de desempenho config
AUTOTUNE = tf.data.AUTOTUNE

train_set = test_set.cache().prefetch(buffer_size = AUTOTUNE)
val_set = val_set.cache().prefetch(buffer_size = AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size = AUTOTUNE)

In [ ]:
#Gerando dados
data_augumentation = tf.keras.models.Sequential()
data_augumentation.add(layers.experimental.preprocessing.RandomFlip('horizontal'))
data_augumentation.add(layers.experimental.preprocessing.RandomFlip('vertical'))
data_augumentation.add(layers.experimental.preprocessing.RandomRotation(0.2))
data_augumentation.add(layers.experimental.preprocessing.RandomZoom(0.2))

In [ ]:
y = []

for x, label in test_set:
    y.append(label.numpy())

In [ ]:
test_labels = []
for i in range(len(y)):
  for j in range(len(y[i])):
    test_labels.append(y[i][j])

# CNN

In [ ]:
#objeto de sequencia de camadas
CNN = tf.keras.models.Sequential()

#Aumento de dados
CNN.add(data_augumentation)

#normalizando os dados em escala [0;1]
CNN.add(layers.Rescaling(1./255))

# #adicionando o primeiro bloco de convolucao e max pooling
CNN.add(layers.Conv1D(128, 3, activation = 'relu'))
CNN.add(layers.MaxPooling2D((2,2)))

#adicionando o segundo bloco de convolucao e max pooling
CNN.add(layers.Conv2D(256, 3, activation = 'relu'))
CNN.add(layers.MaxPooling2D((2,2)))

#adicionando o terceiro bloco de convolucao e max pooling
CNN.add(layers.Conv2D(512, 3, activation = 'relu'))
CNN.add(layers.MaxPooling2D((2,2)))

#adicionando o quarto bloco de convolucao e max pooling
CNN.add(layers.Conv2D(512, 3, activation = 'relu'))
CNN.add(layers.MaxPooling2D((2,2)))

#adicionando o quarto bloco de convolucao e max pooling
CNN.add(layers.Conv2D(1024, 3, activation = 'relu'))
CNN.add(layers.MaxPooling2D((2,2)))

#adicionando o quarto bloco de convolucao e max pooling
CNN.add(layers.Conv2D(1024, 3, activation = 'relu'))
CNN.add(layers.MaxPooling2D((2,2)))

In [ ]:
#Adicionando camadas de classificacao
CNN.add(layers.Flatten())
CNN.add(layers.Dense(1024, activation = 'relu'))
CNN.add(layers.Dropout(0.25))
CNN.add(layers.Dense(1024, activation = 'relu'))
CNN.add(layers.Dropout(0.2))
CNN.add(layers.Dense(17, activation = 'softmax'))

In [ ]:
#ponderacao de classes
class_weight = {0.1, 0.1, 4, 0.2, 2, 0.2, 0.9, 2, 0.9, 0.7, 0.9, 0.4, 10, 0.4, 10, 0.2, 2}
class ClassWeightCallback(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        self.model.optimizer.class_weights = class_weight

In [ ]:
#Compilacao do modelo
CNN.compile(optimizer = 'adam',
                      loss = 'sparse_categorical_crossentropy',
                      metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

reduceLR = ReduceLROnPlateau(monitor='val_loss', patience= 4, verbose= 1, mode='min', factor=  0.2, min_lr = 1e-6)

early_stopping = EarlyStopping(monitor='val_loss', patience = 8 , verbose=1, mode='min', restore_best_weights= True)

checkpoint = ModelCheckpoint('C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/CNN', monitor='val_loss', verbose=1, save_best_only=True, mode= 'min')

callbacks= [reduceLR, early_stopping, checkpoint, ClassWeightCallback()]

In [ ]:
CNN.fit(train_set, validation_data = val_set, epochs = 100, callbacks = [callbacks])

In [ ]:
#Gráfico da accuracia do modelo treinamento e valisacao em relacao as épocas
plt.plot(CNN.history.history['accuracy'])
plt.plot(CNN.history.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper left')

plt.show()

In [ ]:
#Gráfico da erro do modelo treinamento e valisacao em relacao as épocas
plt.plot(CNN.history.history['loss'])
plt.plot(CNN.history.history['val_loss'])
plt.title('Erro do modelo')
plt.ylabel('Erro')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper left')
plt.show()

In [ ]:
import time

t = time.time()

export_path = "C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/CNN/{}".format(int(t))
CNN.save(export_path, save_format = 'tf')

export_path

In [ ]:
CNN.summary()

In [ ]:
from sklearn.metrics import classification_report

predicted_labels = np.argmax(CNN.predict(test_set), axis=1)

print(classification_report(test_labels, predicted_labels, target_names = class_names))

In [ ]:
#Funcao para plotar matriz de confusao
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(actual, predic):

    cm = confusion_matrix(actual, predic)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10,10))
    cmap=plt.cm.Reds
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title('Matriz de confusão', fontsize = 20)

    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation = 90, fontsize = 12)
    plt.yticks(tick_marks, class_names, fontsize = 12)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.2f'),
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black", fontsize = 14)

    plt.ylabel('Classes Verdadeiras', fontsize = 18, color = 'green')
    plt.xlabel('Classes preditadas', fontsize = 18, color = 'red')
    plt.show()

In [ ]:
# plotando a matriz de confusao
plot_confusion_matrix(test_labels, predicted_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

a = accuracy_score(test_labels, predicted_labels)
p = precision_score(test_labels, predicted_labels, average = 'macro')
r = recall_score(test_labels, predicted_labels, average = 'macro')
f = f1_score(test_labels, predicted_labels, average = 'macro')

print("Acuracia: ", a)
print("Precisao: ", p)
print("Recall: ", r)
print("F1-score: ", f)

In [ ]:
#Convertendo o mdelo CNN
converter = tf.lite.TFLiteConverter.from_keras_model(CNN)


converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Salvando o modelo.
with open('C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/CNN/CNNLite.tflite', 'wb') as f:
  f.write(tflite_model)

# MobileNet V2

In [ ]:
IMG_SHAPE = (256, 256) + (3,)
model_MobileNetV2 = tf.keras.applications.MobileNetV2(
    input_shape = IMG_SHAPE,
    include_top = False,
    weights = 'imagenet'
)

model_MobileNetV2.trainable = False

model_MobileNetV2.summary()

In [ ]:
#objeto de sequencia de camadas MobileNet V2
MNet = tf.keras.models.Sequential()

#Aumento de dados
MNet.add(data_augumentation)

#normalizando os dados em escala
MNet.add(layers.Rescaling(1./127.5, offset=-1))
MNet.add(model_MobileNetV2)
MNet.add(layers.GlobalAveragePooling2D())
MNet.add(layers.Flatten())
MNet.add(layers.Dense(1024, activation = 'relu'))
MNet.add(layers.Dense(1024, activation = 'relu'))
MNet.add(layers.Dense(17, activation = 'softmax'))

In [ ]:
#Compilacao do modelo
MNet.compile(optimizer = 'adam',
                      loss = 'sparse_categorical_crossentropy',
                      metrics = ['accuracy'])

In [ ]:
#ponderacao de classes
class_weight = {0.1, 0.1, 4, 0.2, 2, 0.2, 0.9, 2, 0.9, 0.7, 0.9, 0.4, 10, 0.4, 10, 0.2, 2}
class ClassWeightCallback(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        self.model.optimizer.class_weights = class_weight

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

reduceLR = ReduceLROnPlateau(monitor='val_loss', patience= 4, verbose= 1, mode='min', factor=  0.2, min_lr = 1e-6)

early_stopping = EarlyStopping(monitor='val_loss', patience = 8 , verbose=1, mode='min', restore_best_weights= True)

checkpoint = ModelCheckpoint('C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/CNN', monitor='val_loss', verbose=1, save_best_only=True, mode= 'min')

callbacks= [reduceLR, early_stopping, checkpoint, ClassWeightCallback()]

In [ ]:
MNet.fit(train_set, validation_data = val_set, epochs = 100, callbacks = callbacks)

In [ ]:
#Gráfico da accuracia do modelo treinamento e valisacao em relacao as épocas
plt.plot(MNet.history.history['accuracy'])
plt.plot(MNet.history.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper left')

plt.show()

In [ ]:
#Gráfico da erro do modelo treinamento e valisacao em relacao as épocas
plt.plot(MNet.history.history['loss'])
plt.plot(MNet.history.history['val_loss'])
plt.title('Erro do modelo')
plt.ylabel('Erro')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper left')
plt.show()

In [ ]:
import time

t = time.time()

export_path = "C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/MobileNet/{}".format(int(t))
MNet.save(export_path, save_format = 'tf')

export_path

In [ ]:
MNet.summary()

In [ ]:
from sklearn.metrics import classification_report

predicted_labels = np.argmax(MNet.predict(test_set), axis=1)

print(classification_report(test_labels, predicted_labels, target_names = class_names))

In [ ]:
# plotando a matriz de confusao
plot_confusion_matrix(test_labels, predicted_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

a = accuracy_score(test_labels, predicted_labels)
p = precision_score(test_labels, predicted_labels, average = 'macro')
r = recall_score(test_labels, predicted_labels, average = 'macro')
f = f1_score(test_labels, predicted_labels, average = 'macro')

print("Acuracia: ", a)
print("Precisao: ", p)
print("Recall: ", r)
print("F1-score: ", f)

In [ ]:
path_MNet='C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/MobileNet/'
converter = tf.lite.TFLiteConverter.from_saved_model(path_MNet)
tflite_model = converter.convert()
with open('C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/MNLite.tflite', 'wb') as f: f.write(tflite_model)

# VGG 16

In [ ]:
IMG_SHAPE = (256, 256) + (3,)
model_VGG = tf.keras.applications.VGG16(
    input_shape = IMG_SHAPE,
    include_top = False,
    weights = 'imagenet'
)

model_VGG.trainable = False

model_VGG.summary()

In [ ]:
#objeto de sequencia de camadas VGG
modelVGG = tf.keras.models.Sequential()

#Aumento de dados
modelVGG.add(data_augumentation)

#normalizando os dados em escala
modelVGG.add(layers.Rescaling(1./255))
modelVGG.add(model_VGG)
modelVGG.add(layers.GlobalAveragePooling2D())
modelVGG.add(layers.Dense(256, activation = 'relu'))
modelVGG.add(layers.Dense(512, activation = 'relu'))
modelVGG.add(layers.Dropout(0.2))
modelVGG.add(layers.Dense(1024, activation = 'relu'))
modelVGG.add(layers.Dense(1024, activation = 'relu'))
modelVGG.add(layers.Dense(17, activation = 'softmax'))

In [ ]:
class_weight = {0.1, 0.1, 4, 0.2, 2, 0.2, 0.9, 2, 0.9, 0.7, 0.9, 0.4, 10, 0.4, 10, 0.2, 2}
class ClassWeightCallback(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        self.model.optimizer.class_weights = class_weight

In [ ]:
#Compilacao do modelo
modelVGG.compile(optimizer = 'adam',
                      loss = 'sparse_categorical_crossentropy',
                      metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

reduceLR = ReduceLROnPlateau(monitor='val_loss', patience= 4, verbose= 1, mode='min', factor=  0.2, min_lr = 1e-4)

early_stopping = EarlyStopping(monitor='val_loss', patience = 8 , verbose=1, mode='min', restore_best_weights= True)

checkpoint = ModelCheckpoint('C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/VGG', monitor='val_loss', verbose=1, save_best_only=True, mode= 'min')

callbacks= [reduceLR, early_stopping, checkpoint, ClassWeightCallback()]

In [ ]:
modelVGG.fit(train_set, validation_data = val_set, epochs = 100, callbacks = callbacks)

In [ ]:
#Gráfico da accuracia do modelo treinamento e valisacao em relacao as épocas
plt.plot(modelVGG.history.history['accuracy'])
plt.plot(modelVGG.history.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper left')

plt.show()

In [ ]:
#Gráfico da erro do modelo treinamento e valisacao em relacao as épocas
plt.plot(modelVGG.history.history['loss'])
plt.plot(modelVGG.history.history['val_loss'])
plt.title('Erro do modelo')
plt.ylabel('Erro')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper left')
plt.show()

In [ ]:
import time

t = time.time()

export_path = "C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/VGG/{}".format(int(t))
modelVGG.save(export_path, save_format = 'tf')

export_path

In [ ]:
modelVGG.summary()

In [ ]:
from sklearn.metrics import classification_report

predicted_labels = np.argmax(modelVGG.predict(test_set), axis=1)

print(classification_report(test_labels, predicted_labels, target_names = class_names))

In [ ]:
# plot confusion matrix
plot_confusion_matrix(test_labels, predicted_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

a = accuracy_score(test_labels, predicted_labels)
p = precision_score(test_labels, predicted_labels, average = 'macro')
r = recall_score(test_labels, predicted_labels, average = 'macro')
f = f1_score(test_labels, predicted_labels, average = 'macro')

print("Acuracia: ", a)
print("Precisao: ", p)
print("Recall: ", r)
print("F1-score: ", f)

In [ ]:
path_MNet='C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/VGG/'
converter = tf.lite.TFLiteConverter.from_saved_model(path_MNet)
tflite_model = converter.convert()
with open('C:/Users/DREY/OneDrive/Documents/Moda/Notebooks/Models/VGGLite.tflite', 'wb') as f: f.write(tflite_model)